In [1]:
%matplotlib notebook

import os
import glob
from matplotlib import pyplot as plt
from matplotlib import colors
import numpy as np
import pandas as pd
import piff
from piff import Star
import fitsio
from piff.des.decaminfo import DECamInfo
dinfo = DECamInfo()

from esutil.htm import HTM
hmat = HTM()

In [2]:
def fillOpticsParams(piff_files,exposures,chi2_files,outfile):
    # get the Optics fitted parameters

    # output file
    outfits = fitsio.FITS(outfile,'rw',clobber=True)
    
    # build parameter names
    allvarnames = ['L0','size','g1','g2']
    for iZp in range(4,37+1):
        for iZf in range(1,3+1):
            zvarname = 'zPupil%03d_zFocal%03d' % (iZp,iZf)
            allvarnames.append(zvarname)
    
    # use first file to find the floating variables
    fits = fitsio.FITS(piff_files[0])
    psf_solution = fits['psf_solution'].read()

    # selected floating parameters
    float_vars = []    
    for colname in allvarnames:
        isfixn = 'fix_' + colname
        if not psf_solution[isfixn]:
            float_vars.append(colname)
            float_vars.append("error_"+colname)

    # names,types
    dtype_tuple = []
    dtype_tuple.append(('expnum','i8'))
    dtype_tuple.append(('chi2','f8'))
    for avar in float_vars:
        dtype_tuple.append((avar,'f8'))

    # loop over all files
    for i,afile in enumerate(piff_files):
        
        if i==20*int(i/20):
            print("File # ",i)
                    
        # output record array
        outdata = np.zeros(1, dtype=dtype_tuple)            
            
        # open file
        fits = fitsio.FITS(afile)
        psf_solution = fits['psf_solution'].read()

        # pull out desired values
        float_values = fits['psf_solution'].read(columns=float_vars)
        
        for avar in float_vars:
            outdata[avar] = float_values[avar]
            
        # chi2 live in a different file...
        chi_i = np.load(chi2_files[i])
        chisq= np.sum(np.square(chi_i))
        outdata['chi2'] = chisq
        
        # add the exposure number...
        expnum = exposures[i]
        outdata['expnum'] = expnum

        # write out
        if i==0:
            # create a new table extension and write the data
            outfits.write(outdata,extname='fitparams')
        else:
            outfits[-1].append(outdata)
            
    # done
    outfits.close()

In [3]:
# now get data from all the Train stars - from both .piff and .h5 files
# 
# we want x,y,u,v,chipnum,ra,dec,shape[3:5],shape_error[3:5],fov_x,fov_y from .piff
# and model_e0,model_e1,model_2, optics_size, optics_g1, optics_g2, atmo_size, atmo_g1, atmo_g2 from .h5

def concat_stars(piff_files,h5_files,outfile,
                 piffcols = ['x','y','u','v','chipnum','ra','dec',
                             'e0','e1','e2','se0','se1','se2', # variables in shape
                             'fov_x','fov_y','pix_area', # variables calculated from others
                             'atmo_size','atmo_g1','atmo_g2','optics_L0','optics_size','optics_g1','optics_g2'],  # variables in params 0 to 6
                 h5cols=['model_e0', 'model_e1','model_e2']):
    """ from a list of .piff files, collect star information and concatenate into a single file
    """
    
    # output variables
    dtype_tuple = []
    for avar in piffcols:
        dtype_tuple.append((avar,'f8'))
    for avar in h5cols:
        dtype_tuple.append((avar,'f8'))
        
        
    # output file
    outfits = fitsio.FITS(outfile,'rw',clobber=True)
            
    for i,f in enumerate(piff_files):
        if i==20*np.int(i/20): print('Loading file {0} of {1}'.format(i, len(piff_files)))
        fits = fitsio.FITS(f)
        h5f = pd.read_hdf(h5_files[i])  # assume files are ordered the same!!
            
        # use only Test stars from .piff file
        extnames = 'psf_stars'
        d = fits[extnames].read()
        piff_names = d.dtype.names
        
        # match the stars in the h5 file (it has more stars)
        u_h5 = h5f['u']
        v_h5 = h5f['v']
        u_piff = d['u'][:]
        v_piff = d['v'][:]

        # use idx_to_h5
        idx_to_piff,idx_to_h5,dista = hmat.match(u_piff,v_piff,u_h5,v_h5,2./3600.,maxmatch=1)
        h5fc = h5f.iloc[idx_to_h5]
        
                
        # setup output recarray
        nrows = d.shape[0]
        outdata = np.zeros(nrows, dtype=dtype_tuple)
    
        for var in piffcols:
            if var in piff_names:
                outdata[var] = d[var]
            else:
                # build the variable from shape or params
                if var == 'e0':
                    outdata[var] = d['shape'][:,3]
                elif var == 'e1':
                    outdata[var] = d['shape'][:,4]
                elif var == 'e2':
                    outdata[var] = d['shape'][:,5]
                elif var == 'se0':
                    outdata[var] = d['shape_error'][:,3]
                elif var == 'se1':
                    outdata[var] = d['shape_error'][:,4]
                elif var == 'se2':
                    outdata[var] = d['shape_error'][:,5]
                elif var == 'fov_x':                    
                    ix = d['x'][:] - 0.5
                    iy = d['y'][:] - 0.5
                    chipnums = d['chipnum'][:].astype(int)
                    fov_x,fov_y = dinfo.getPosition_chipnum(chipnums, ix, iy)
                    factor = (0.015/0.263) 
                    fov_x /= factor
                    fov_y /= factor                    
                    outdata[var] = fov_x
                elif var == 'fov_y':                    
                    ix = d['x'][:] - 0.5
                    iy = d['y'][:] - 0.5
                    chipnums = d['chipnum'][:].astype(int)
                    fov_x,fov_y = dinfo.getPosition_chipnum(chipnums, ix, iy)
                    factor = (0.015/0.263) 
                    fov_x /= factor
                    fov_y /= factor                    
                    outdata[var] = fov_y
                elif var == 'pix_area':
                    pixsiz = np.abs((d['dudx'][:] * d['dvdy'][:]) - (d['dudy'][:] * d['dvdx'][:]))
                    outdata[var] = pixsiz
                elif var == 'atmo_size':
                    try:      # d['params'] only present with single star fits, not optics fit alone - improve this kludge...
                        outdata[var] = d['params'][:,0]
                    except:
                        pass
                elif var == 'atmo_g1':
                    try:
                        outdata[var] = d['params'][:,1]                        
                    except:
                        pass
                elif var == 'atmo_g2':
                    try:
                        outdata[var] = d['params'][:,2]                    
                    except:
                        pass
                elif var == 'optics_L0':
                    try:
                        outdata[var] = d['params'][:,3]                    
                    except:
                        pass
                elif var == 'optics_size':
                    try:
                        outdata[var] = d['params'][:,4]
                    except:
                        pass
                elif var == 'optics_g1':
                    try:
                        outdata[var] = d['params'][:,5]
                    except:
                        pass
                elif var == 'optics_g2':
                    try:
                        outdata[var] = d['params'][:,6]                        
                    except:
                        pass
             
        # h5 has more stars than the .piff, need to match them!
        for var in h5cols:
            outdata[var] = h5fc[var]

                    
                    
        # write out
        if i==0:
            # create a new table extension and write the data
            outfits.write(outdata,extname='psf_stars')
        else:
            outfits[-1].append(outdata)
            
    # done
    outfits.close()
                    
                    

In [4]:
def mkStarSummary(directory,outpfile,outfile,psf_name='psf_optatmo',nfilemax=None):
    # directory: where Piff output resides
    # outpfile: optics parameters per image
    # outfile: star parameters for all stars/images
    
    # get files
    #psf_name = "psf_optatmo"
    very_original_files = sorted(glob.glob('{0}/*/{1}.piff'.format(directory, psf_name)))

    print("Number of optatmo fits found = ",len(very_original_files))
    
    # filter out acceptable fits
    original_files = []
    original_exposures = []
    try:
        acceptable_exposures = np.load("{0}/acceptable_exposures.npy".format(directory))
        for very_original_file in very_original_files:
            very_original_exposure = very_original_file.split("/")[-2][2:]
            if very_original_exposure in acceptable_exposures:
                original_files.append(very_original_file)
                original_exposures.append(very_original_exposure)
    except:
        for very_original_file in very_original_files:
            very_original_exposure = very_original_file.split("/")[-2][2:]
            original_files.append(very_original_file)
            original_exposures.append(very_original_exposure)        
        
    print("Number of acceptable fits found = ",len(original_files))
    
    # get h5 file names corresponding to these
    h5_original_files = []
    for afile in original_files:
        thedir = os.path.dirname(afile)
        h5_original_files.append(os.path.join(thedir,"shapes_train_psf_optatmo_opt_only.h5"))
    print("Number of h5 files = ",len(h5_original_files))
    
    # get file with Chi2 values corresponding to these...
    chi2_original_files = []
    for afile in original_files:
        thedir = os.path.dirname(afile)
        chi2_original_files.append(os.path.join(thedir,"full_optical_chisq_vector.npy"))
    print("Number of chi2 npy files = ",len(chi2_original_files))
       
    
    if nfilemax==None:
        # write out all optics fit parameters
        fillOpticsParams(original_files,original_exposures,chi2_original_files,outpfile)

        # concatenate .piff and .h5 file information
        concat_stars(original_files,h5_original_files,outfile)
        
    else:
        # write out all optics fit parameters
        fillOpticsParams(original_files[0:nfilemax],original_exposures[0:nfilemax],chi2_original_files[0:nfilemax],outpfile)

        # concatenate .piff and .h5 file information
        concat_stars(original_files[0:nfilemax],h5_original_files[0:nfilemax],outfile)        
        

In [ ]:
directory = "/nfs/slac/kipac/fs1/g/des/aresh/old_fit_checks_and_other_stuff/fit_checks_round36/dummy/y1_500_exposure_fit_check_band_i_shape_mode_round2"
outpfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_round2_fitparams.fits"
outfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_round2.fits"
mkStarSummary(directory,outpfile,outfile)

In [ ]:
# test parameters file
# OptAtmo fits
outp = fitsio.FITS(outpfile,'r')
varnames = outp['fitparams'].get_colnames()
print(varnames)
print(len(varnames))
t = outp['fitparams'].read()

In [ ]:
#plots
f,axa = plt.subplots(6,4,figsize=(10,10))
ax = axa.flatten()

i = 0
for avar in varnames:
    if not 'error' in avar:
        h = ax[i].hist(t[avar],bins=50)
        ax[i].set_xlabel(avar)
        i = i + 1
    
f.tight_layout()

In [ ]:
# other runs...
directory = "/nfs/slac/kipac/fs1/g/des/aresh/y1_500_exposure_fit_check_band_i_shape_mode_base_case"
outpfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_basecase_fitparams.fits"
outfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_basecase.fits"
mkStarSummary(directory,outpfile,outfile,psf_name='psf_optatmo_opt_only')

In [ ]:
directory = "/nfs/slac/kipac/fs1/g/des/aresh/y1_500_exposure_fit_check_band_i_shape_mode_scipy_dog_leg_for_main_optical_fit"
outpfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_dogleg_fitparams.fits"
outfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_dogleg.fits"
mkStarSummary(directory,outpfile,outfile,psf_name='psf_optatmo_opt_only')

In [5]:
directory = "/nfs/slac/kipac/fs1/g/des/aresh/y1_one_exposure_fit_check_band_i_shape_mode_class_based_WF_no_lin_coma_terms"
outpfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_nolincoma_fitparams.fits"
outfile = "/nfs/slac/g/ki/ki06/roodman/Piff/y1_500_i_shape_mode_nolincoma.fits"
mkStarSummary(directory,outpfile,outfile,psf_name='psf_optatmo_opt_only')

Number of optatmo fits found =  499
Number of acceptable fits found =  499
Number of h5 files =  499
Number of chi2 npy files =  499
File #  0
File #  20
File #  40
File #  60
File #  80
File #  100
File #  120
File #  140
File #  160
File #  180
File #  200
File #  220
File #  240
File #  260
File #  280
File #  300
File #  320
File #  340
File #  360
File #  380
File #  400
File #  420
File #  440
File #  460
File #  480
Loading file 0 of 499
Loading file 20 of 499
Loading file 40 of 499
Loading file 60 of 499
Loading file 80 of 499
Loading file 100 of 499
Loading file 120 of 499
Loading file 140 of 499
Loading file 160 of 499
Loading file 180 of 499
Loading file 200 of 499
Loading file 220 of 499
Loading file 240 of 499
Loading file 260 of 499
Loading file 280 of 499
Loading file 300 of 499
Loading file 320 of 499
Loading file 340 of 499
Loading file 360 of 499
Loading file 380 of 499
Loading file 400 of 499
Loading file 420 of 499
Loading file 440 of 499
Loading file 460 of 499
Loa